## [groqCloud](https://console.groq.com/)

In [78]:
import getpass  # Import the getpass module to securely handle password prompts
import os  # Import the os module to interact with the operating system's environment variables

# Check if the environment variable "GROQ_API_KEY" is not already set
if "GROQ_API_KEY" not in os.environ:
    # Prompt the user to enter their Groq API key and set it as an environment variable
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")


In [56]:
from langchain_groq import ChatGroq  # Importing the ChatGroq class from the langchain_groq module

# Initializing the ChatGroq model with specific parameters
llm = ChatGroq(
    model="llama-3.1-70b-versatile",  # Specifying the model to be used
    temperature=0  # Setting the temperature to 0 for deterministic output
)

In [58]:
# Invoke the language model (LLM) with a query to determine which LLM is being used
response = llm.invoke("Which LLM you are using?")

# Print the content of the response received from the LLM
print(response.content)

I'm using a large language model, but I don't have have access to real-time information about the specific model I'm using. My knowledge was last updated in December 2023, and I don't have information about any updates or changes made after that date.


## [WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base/)

In [60]:
# Importing the WebBaseLoader class from the langchain_community.document_loaders module
from langchain_community.document_loaders import WebBaseLoader

# Creating an instance of WebBaseLoader to load the job posting from the specified URL
loader = WebBaseLoader("https://jobs.nike.com/job/R-32222")  

In [62]:
# Load the data using the loader and retrieve the last item from the loaded data
# The 'pop()' method removes and returns the last item from the list
page_data = loader.load().pop().page_content

# Print the content of the page data to the console
print(page_data)

Apply for Senior Machine Learning Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Machine Learning EngineerGdansk, PomeranianBecome a Part of the NIKE, I

## Prompt

In [64]:
from langchain_core.prompts import PromptTemplate

# Create a prompt template for extracting job postings from scraped website data
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from a career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing following keys:
    'role', 'skills', and 'description'.
    Only return the valid JSON.
    ### Valid JSON (NO PREAMBLE):
    """
)

# Chain the prompt with the language model (llm) to create an extraction process
chain_extract = prompt_extract | llm

# Invoke the chain with the input data containing the scraped page data
res = chain_extract.invoke(input = {'page_data': page_data})

# Print the content of the response, which should be the extracted job postings in JSON format
print(res.content)

```json
{
  "role": "Senior Machine Learning Engineer",
  "skills": [
    "3+ years of experience in the field of ML Engineering or Software Engineering",
    "Bachelor’s degree in a relevant field, like computer science or software engineering",
    "Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production",
    "Ability to write robust, maintainable, and extendable code in Python",
    "Experience working in and/or collaborating with a partial or fully distributed team",
    "Strong experiential understanding of data structures, algorithms, and data solutions",
    "Familiarity with frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI or similar platforms and frameworks",
    "Experience with cloud architecture and technologies (preferably Amazon Web Services: ECR, SageMaker, Lambda, API Gateway)",
    "Familiarity with pipeline orchestration tools such as AirFlow",
    "Awareness of CI/CD pipelines and containerization

In [66]:
from langchain_core.output_parsers import JsonOutputParser

# Create an instance of JsonOutputParser to handle JSON output
json_parser = JsonOutputParser()

# Parse the content of the response (res.content) into a JSON format
json_res = json_parser.parse(res.content)

# Display the parsed JSON result
json_res

{'role': 'Senior Machine Learning Engineer',
 'skills': ['3+ years of experience in the field of ML Engineering or Software Engineering',
  'Bachelor’s degree in a relevant field, like computer science or software engineering',
  'Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
  'Ability to write robust, maintainable, and extendable code in Python',
  'Experience working in and/or collaborating with a partial or fully distributed team',
  'Strong experiential understanding of data structures, algorithms, and data solutions',
  'Familiarity with frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI or similar platforms and frameworks',
  'Experience with cloud architecture and technologies (preferably Amazon Web Services: ECR, SageMaker, Lambda, API Gateway)',
  'Familiarity with pipeline orchestration tools such as AirFlow',
  'Awareness of CI/CD pipelines and containerization'],
 'description': 'As a Sr. Machi

In [68]:
import pandas as pd  # Import the pandas library for data manipulation and analysis

df = pd.read_excel("portfolio.xlsx")  # Read the Excel file "portfolio.xlsx" into a DataFrame
df  # Display the DataFrame

,techstack,links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Python, Django, MySQL",https://example.com/python-portfolio
2,"DevOps, Jenkins, Docker",https://example.com/devops-portfolio
3,"Machine Learning, Python",https://example.com/ml-python-portfolio
4,"Generative AI, LLM",https://example.com/genai-llm-portfolio
5,"Deep learning, Python, , TensorFlow",https://example.com/dl-python-portfolio


In [70]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')  # Initialize a persistent client for the ChromaDB vector store
collection = client.get_or_create_collection(name="portfolio")  # Create or retrieve a collection named "portfolio"

if not collection.count():  # Check if the collection is empty
    for _, row in df.iterrows():  # Iterate over each row in the DataFrame 'df'
        collection.add(documents=row["techstack"],  # Add the 'techstack' from the current row as a document
                       metadatas={"links": row["links"]},  # Add the 'links' from the current row as metadata
                       ids=[str(uuid.uuid4())])  # Generate a unique ID for the document using uuid and add it to the collection

In [72]:
# Query the collection for the specified skills in json_res
# Retrieve the top 2 results based on the query
# Extract the 'metadatas' field from the results, defaulting to an empty list if not found
links = collection.query(query_texts=json_res['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/dl-python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/dl-python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/dl-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/dl-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/genai-llm-portfolio'}],
 [{'links': 'https://example.com/dl-python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/dl-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/dl-python-portfolio

## email generator

In [76]:
# Create a prompt template for generating a cold email
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are ABC, a business development executive at XYZ. XYZ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase XYZ's portfolio: {link_list}
        Remember you are ABC, BDE at XYZ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

# Create a chain that combines the prompt with a language model (llm) for email generation
chain_email = prompt_email | llm

# Invoke the chain with the job description and links to generate the email content
res = chain_email.invoke({"job_description": str(json_res), "link_list": links})

# Print the generated email content
print(res.content)

Subject: Expert Machine Learning Engineering Solutions for Nike's AIML Team

Dear Hiring Manager,

I came across the Senior Machine Learning Engineer role at Nike and was impressed by the company's commitment to leveraging AI and machine learning to drive business growth. As a Business Development Executive at XYZ, I'd like to introduce our company as a trusted partner in delivering cutting-edge machine learning engineering solutions.

Our team at XYZ has extensive experience in developing sophisticated analytics systems, working with cross-disciplinary teams, and creating high-quality solutions that power businesses like yours. We've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our expertise aligns perfectly with the requirements of the Senior Machine Learning Engineer role, including:

* Developing robust, maintainable, and extendable code in Python
* Experience with frameworks